In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("ChartbustersParticipantsData\Data_Train.csv",infer_datetime_format=True,encoding='utf-8')

In [3]:
df_test = pd.read_csv("ChartbustersParticipantsData\Data_Test.csv",infer_datetime_format=True,encoding='utf-8')

In [4]:
df_test['Views']=None

In [5]:
df.Name = df.Name.str.extract('([0-9a-zA-Z ]*)')
df_test.Name = df_test.Name.str.extract('([0-9a-zA-Z ]*)')

In [6]:
data = pd.concat([df,df_test],axis=0,sort=False,ignore_index=True)

In [7]:
data.index = data.Unique_ID

In [8]:
data.drop(columns=['Unique_ID'],inplace=True)

In [9]:
data.Name = data.Name.str.extract('([0-9a-zA-Z ]*)')

In [10]:
data.drop(columns=['Country','Song_Name'],inplace=True)

In [11]:
data.Views = pd.to_numeric(data.Views)

In [12]:
for cols in ['Likes','Popularity']:
    data[cols] = data[cols].apply(lambda x: x.replace(",",''))

In [13]:
data['Likes']=np.where(data['Likes'].str[-1]=='K',pd.to_numeric(data['Likes'].str[0:-1])*1000,data['Likes'])
data['Likes']=np.where(data['Likes'].str[-1]=='M',pd.to_numeric(data['Likes'].str[0:-1])*1000000,data['Likes'])
data['Popularity']=np.where(data['Popularity'].str[-1]=='M',pd.to_numeric(data['Popularity'].str[0:-1])*1000000,data['Popularity'])
data['Popularity']=np.where(data['Popularity'].str[-1]=='K',pd.to_numeric(data['Popularity'].str[0:-1])*1000,data['Popularity'])

In [14]:
data['Timestamp']= pd.to_datetime(data['Timestamp'])

In [15]:
latest = data['Timestamp'].max()

In [16]:
data['Timestamp'] = (latest - data['Timestamp'] ).astype('timedelta64[D]')

In [43]:
cat_cols = pd.get_dummies(data.select_dtypes(exclude=np.number))

In [20]:
data.Timestamp = data.Timestamp.astype(int)
data.Likes = data.Likes.astype(int)
data.Popularity = data.Popularity.astype(int)

In [21]:
data['Followers_Per_Likes'] = np.where(data.Likes>0,data.Followers/data.Likes,0)

In [22]:
data['Followers_Per_Comments'] = np.where(data.Comments>0,data.Followers/data.Comments,0)

In [23]:
data['Likes_Per_Comments'] = np.where(data.Comments>0,data.Likes/data.Comments,0)

In [24]:
data['Songs_Cnt'] = data["Name"].map(data["Name"].value_counts())

In [32]:
data['Zero_Likes'] = np.where(data.Likes==0,1,0)
data['High_Likes'] = np.where(data.Likes>100000,1,0)

In [33]:
data['Zero_Comments'] = np.where(data.Comments==0,1,0)

In [34]:
data['Zero_Popularity'] = np.where(data.Popularity==0,1,0)

In [35]:
data['Likes_Per_Timestamp'] = np.where(data.Timestamp>0,data.Likes/data.Timestamp,0)

In [36]:
data['LikesnPopularity'] = data['Likes'] * data['Popularity']

In [37]:
data['Likes_Per_Popularity'] =  np.where(data.Popularity>0,data.Likes/data.Popularity,0)

In [38]:
data['Mean_Popularity'] = data['Name'].map(data.groupby('Name').Popularity.mean())

In [39]:
data['Avg_Likes'] = data['Name'].map(data.groupby('Name').Likes.mean())

In [40]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [41]:
drop_cols = ["Name_"+ name for name in df.Name.unique().tolist() if name not in df_test.Name.unique().tolist()]

In [44]:
cat_cols.drop(columns = drop_cols,inplace=True)

In [45]:
X = pd.concat([cat_cols,data.select_dtypes(include=np.number)],axis=1)

In [46]:
testX = X[X['Views'].isna()]

In [47]:
trainX = X[X['Views'].notna()]

In [48]:
trainY = trainX.pop('Views').astype(int)

In [49]:
testX.pop('Views')

Unique_ID
562546    NaN
907584    NaN
213013    NaN
340312    NaN
41854     NaN
           ..
183403    NaN
1272578   NaN
539454    NaN
1543013   NaN
415046    NaN
Name: Views, Length: 19615, dtype: float64

In [50]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso,LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

In [51]:
train_x, test_x, train_y,test_y = train_test_split(trainX,trainY,test_size=0.2, random_state=5)

In [52]:
print(train_x.shape,train_y.shape,test_x.shape,test_y.shape)

(62766, 1116) (62766,) (15692, 1116) (15692,)


In [193]:
lin = LassoCV(n_alphas=['0.1','1','10','0.01'])

In [248]:
lin = Lasso(alpha=0.1)
lin.fit(train_x,train_y)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.867720242973544e+16, tolerance: 90868133607135.53
  positive)


Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [249]:
print("Train RMSE:",np.sqrt(mean_squared_error(train_y,lin.predict(train_x))))

Train RMSE: 1583315.3941994915


In [250]:
print("Test RMSE:",np.sqrt(mean_squared_error(test_y,lin.predict(test_x))))

Test RMSE: 2063732.3987288144


In [320]:
train_x.Timestamp.astype(int)

Unique_ID
212750     1299
574389      501
662009      999
962549        1
530696      489
           ... 
220974      676
1321506     216
448409     1877
212546      717
972679       19
Name: Timestamp, Length: 62766, dtype: int32

In [53]:
def model_fit(model,trainx,trainy,testx,testy):
    model.fit(trainx,trainy)
    trainypred = model.predict(trainx)
    print("Train RMSE:",np.sqrt(mean_squared_error(trainy,trainypred)))
    testypred = model.predict(testx)
    print("Test RMSE:",np.sqrt(mean_squared_error(testy,testypred)))
    #return testypred
    

In [54]:
def model_predict(model,trainx,trainy,testx):
    model.fit(trainx,trainy)
    trainypred = model.predict(trainx)
    print("Train RMSE:",np.sqrt(mean_squared_error(trainy,trainypred)))
    testypred = model.predict(testx)
    return testypred
    

In [55]:
def write_results(ytest,file):
    result_df= pd.DataFrame([ytest]).T
    result_df.columns=['Views']
    result_df.index = testX.index
    writer = ExcelWriter(file+'.xlsx')
    result_df.to_excel(writer,index=True)
    writer.save()

In [62]:
rfr = RandomForestRegressor(random_state=5,n_estimators=50)

In [63]:
model_fit(rfr,train_x,train_y,test_x,test_y)

Train RMSE: 283370.5586063274
Test RMSE: 774083.6237148851


In [51]:
from pandas import ExcelWriter

In [270]:
write_results(ytest,"RandomForest")

In [159]:
ytest = model_predict(rfr,trainX,trainY,testX)
write_results(ytest,"RandomForest2")

Train RMSE: 293329.6797670654


In [203]:
for i in range(10,100,10):
    rfr = RandomForestRegressor(random_state=5,n_estimators=i)
    print("n_estimators ={} :".format(i))
    model_fit(rfr,train_x,train_y,test_x,test_y)
    

n_estimators =10 :
Train RMSE: 338793.64673769096
Test RMSE: 747242.148907676
n_estimators =20 :
Train RMSE: 313754.0732218443
Test RMSE: 723959.5011887817
n_estimators =30 :


KeyboardInterrupt: 

In [233]:
["Name_"+name for name in df_test.Name.unique().tolist() if name not in df.Name.unique().tolist()]

['Name_Alisson Vitor',
 'Name_Dj J.Fernandez',
 'Name_Kyle Murdock 1',
 'Name_ISCOO',
 'Name_عمرو',
 'Name_DJ Farrukh',
 'Name_rizky.rilos',
 'Name_Marvel Boy',
 'Name_Cabuizee VIPs',
 'Name_Nguyễn Công Danh']

In [331]:
feat_imp = rfr.feature_importances_

In [332]:
feat_df = pd.concat([pd.DataFrame(testX.columns),pd.DataFrame(feat_imp)],axis=1)

In [333]:
feat_df.columns=['Column','Importance']

In [342]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(16)

In [185]:
for i in range(10,20,2):
    rfr = RandomForestRegressor(random_state=5,n_estimators=40,max_features=i)
    imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(20)
    rfr.fit(train_x[imp_cols],train_y)
    testypred=rfr.predict(test_x[imp_cols])
    print("i={}:,Test RMSE:{}".format(i,np.sqrt(mean_squared_error(test_y,testypred))))
    

i=10:,Test RMSE:719471.9348078237
i=12:,Test RMSE:730722.0337373853
i=14:,Test RMSE:744698.5829642663
i=16:,Test RMSE:734040.660981099
i=18:,Test RMSE:712628.4320706375


In [345]:
rfr = RandomForestRegressor(random_state=5,n_estimators=100,max_features=16)
ytest = model_predict(rfr,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"RandomForest_feat16")

Train RMSE: 265162.463113195


In [187]:
x_scaled = MinMaxScaler().fit_transform(X[imp_cols])

In [188]:
train_x,test_x,train_y,test_y = train_test_split(x_scaled[:trainX.shape[0],:],trainY,test_size=0.2,random_state=5)

In [189]:
train_x.shape

(62766, 20)

In [190]:
train_y.shape

(62766,)

In [335]:
feat_df.sort_values(by=['Importance'],ascending=False)

,Column,Importance
1099,Comments,0.227380
1101,Popularity,0.192404
1100,Likes,0.149666
1110,Likes_Per_Timestamp,0.086726
1111,LikesnPopularity,0.065085
...,...,...
556,Name_Marvel Boy,0.000000
633,Name_Nu Elect Remix,0.000000
689,Name_Prince Winston Thee Third,0.000000
882,Name_Tuan Lux,0.000000


In [61]:
knn = KNeighborsRegressor(n_neighbors=5)

In [192]:
for i in range(100,120,10):
    knn = KNeighborsRegressor(n_neighbors=i)
    #imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(60)
    knn.fit(train_x[imp_cols],train_y)
    testypred=knn.predict(test_x[imp_cols])
    print("i={}:,Test RMSE:{}".format(i,np.sqrt(mean_squared_error(test_y,testypred))))

i=250:,Test RMSE:3268262.0628176234


KeyboardInterrupt: 

In [70]:
from xgboost import XGBRegressor

In [71]:
xgb = XGBRegressor()

In [76]:
trainX['Name_Major Lazer'] = trainX['Name_Major Lazer [OFFICIAL]'

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [340]:

xgb = XGBRegressor(random_state=5,n_estimators=200)
ytest = model_predict(xgb,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"XGBoost3")

C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[04:06:39] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Train RMSE: 328611.71695205546


In [179]:
for i in range(4,7):
    xgb = XGBRegressor(random_state=5,n_estimators=500,max_depth=i)
    xgb.fit(train_x[imp_cols],train_y)
    testypred=xgb.predict(test_x[imp_cols])
    print("i={}:,Test RMSE:{}".format(i,np.sqrt(mean_squared_error(test_y,testypred))))
    
    

C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:28:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
i=4:,Test RMSE:681832.7582993128


C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:29:38] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
i=5:,Test RMSE:648465.7880878866


C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[01:31:26] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
i=6:,Test RMSE:640780.9182854426


In [341]:
feat_df[feat_df.Importance>0].sort_values(by=['Importance'],ascending=False).head(60)

,Column,Importance
1099,Comments,0.227380
1101,Popularity,0.192404
1100,Likes,0.149666
1110,Likes_Per_Timestamp,0.086726
1111,LikesnPopularity,0.065085
1103,Followers_Per_Likes,0.036025
1112,Likes_Per_Popularity,0.034822
1113,Mean_Popularity,0.031899
1105,Likes_Per_Comments,0.026409
1104,Followers_Per_Comments,0.024026


In [182]:
np.sqrt(62766)

250.53143515335555

In [198]:
from sklearn.tree import DecisionTreeRegressor

In [337]:
dtree = DecisionTreeRegressor(random_state=5)

In [338]:
ytest = model_predict(dtree,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"DTree1")

Train RMSE: 380.687606461636


In [203]:
for i in range(100,200,10):
    dtr = DecisionTreeRegressor(random_state=5,max_features=i)
    imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(200)
    dtr.fit(train_x[imp_cols],train_y)
    testypred=dtr.predict(test_x[imp_cols])
    print("i={}:,Test RMSE:{}".format(i,np.sqrt(mean_squared_error(test_y,testypred))))

i=100:,Test RMSE:1052611.77556907
i=110:,Test RMSE:831309.8801806642
i=120:,Test RMSE:947954.0691464593
i=130:,Test RMSE:875987.2170037951
i=140:,Test RMSE:901723.0962989145
i=150:,Test RMSE:854686.4359828248
i=160:,Test RMSE:841042.1793302885
i=170:,Test RMSE:643069.8942032019
i=180:,Test RMSE:1055323.3037450812
i=190:,Test RMSE:764044.1983627678


In [231]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(20)

In [208]:
from sklearn.ensemble import VotingRegressor

In [211]:
dtree = DecisionTreeRegressor(random_state=5,max_features=20)
rfr = RandomForestRegressor(random_state=5,n_estimators=40,max_features=60)
voting = VotingRegressor(estimators=[
('DTree',dtree),('RanFor',rfr)
])

In [212]:
ytest = model_predict(voting,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"Voting")

Train RMSE: 153981.5900797446


In [346]:
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor

In [351]:
ada = AdaBoostRegressor(random_state=5,n_estimators=5)

In [352]:
ytest = model_predict(ada,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"AdaBoost")

Train RMSE: 1008095.8826570979


In [226]:
gbr = GradientBoostingRegressor(random_state=5,max_features=140,n_estimators=140)

In [227]:
ytest = model_predict(gbr,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"GradientBoost")

Train RMSE: 399194.9845537161


In [225]:
for i in range(110,150,10):
    dtr = GradientBoostingRegressor(random_state=5,max_features=100,n_estimators=i)
    imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(200)
    dtr.fit(train_x[imp_cols],train_y)
    testypred=dtr.predict(test_x[imp_cols])
    print("i={}:,Test RMSE:{}".format(i,np.sqrt(mean_squared_error(test_y,testypred))))

i=110:,Test RMSE:773750.0993713099
i=120:,Test RMSE:764735.3725529934
i=130:,Test RMSE:762679.6054679636
i=140:,Test RMSE:757788.9236285165


In [224]:
imp_cols

982              Popularity
981                   Likes
993    Likes_Per_Popularity
991     Likes_Per_Timestamp
986      Likes_Per_Comments
               ...         
224               Name_Eric
804            Name_Virtual
317            Name_Hussein
618              Name_ROJAS
42                Name_Amin
Name: Column, Length: 200, dtype: object

In [228]:
from sklearn.preprocessing import PolynomialFeatures

In [229]:
poly = PolynomialFeatures(degree=2)

In [246]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(20)
trainXpoly = poly.fit_transform(trainX[imp_cols])

In [247]:
testXpoly = poly.transform(testX[imp_cols])

In [234]:
lin = LinearRegression()

In [238]:
ytest = model_predict(rfr,trainXpoly,trainY,testXpoly)
write_results(ytest,"Polynomial_RFR")

Train RMSE: 289047.50823923585


In [239]:
from sklearn

In [240]:
bnb = BernoulliNB()

In [241]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(20)
DT = DecisionTreeRegressor(random_state=5, max_features=20)
DT.fit(trainX[imp_cols],trainY)


DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=20,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=5, splitter='best')

In [242]:
testy_DT = DT.predict(testX[imp_cols])

In [243]:
rfr = RandomForestRegressor(random_state=5,n_estimators=40)

In [244]:
imp_cols=feat_df.sort_values(by=['Importance'],ascending=False)['Column'].head(60)
rfr.fit(trainX[imp_cols],trainY)
testy_RF = rfr.predict(testX[imp_cols])

In [248]:
gbr= GradientBoostingRegressor(random_state=5)
gbr.fit(trainXpoly,trainY)
testy_GBR = rfr.predict(testXpoly)

ValueError: Number of features of the model must match the input. Model n_features is 60 and input n_features is 231 

In [250]:
ensemble = np.mean([testy_DT
                              ,testy_RF
                             ], axis = 0)

In [251]:
ensemble

array([214042.9875,  30803.2875,  15332.225 , ...,   4142.    ,
       101363.35  ,  21217.4875])

In [253]:
result_df = pd.DataFrame(ensemble)

In [254]:
result_df.columns=['Views']
result_df.index = testX.index

In [257]:
writer = ExcelWriter('MeanTree.xlsx')
result_df.to_excel(writer,index=True)

In [258]:
result_df

,Views
Unique_ID,
562546,214042.9875
907584,30803.2875
213013,15332.2250
340312,32708.9875
41854,8542.1875
...,...
183403,65378.4500
1272578,99054.7500
539454,4142.0000


In [259]:
rfr = RandomForestRegressor(random_state=5,n_estimators=1000,max_features=0.1)
ytest = model_predict(rfr,trainX,trainY,testX)
write_results(ytest,"RFR")

KeyboardInterrupt: 

In [356]:
xgb1 = XGBRegressor()
parameters = {'nthread':[2], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [100,200,500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 5,
                        n_jobs = 2,
                        verbose=True)

xgb_grid.fit(trainX[imp_cols],
         trainY)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed: 16.4min
[Parallel(n_jobs=2)]: Done 135 out of 135 | elapsed: 46.3min finished
C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


0.9692657134175081
{'colsample_bytree': 0.7, 'learning_rate': 0.07, 'max_depth': 5, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 2, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


In [354]:
from sklearn.model_selection import GridSearchCV

In [357]:
xgbr = XGBRegressor(colsample_bytree= 0.7, learning_rate= 0.07, max_depth=5, min_child_weight= 4, n_estimators= 500, nthread= 2, objective= 'reg:linear', silent= 1, subsample= 0.7)

In [359]:
xgbr.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=5, eval_metric="rmse",verbose=True)

[0]	validation_0-rmse:3.96394e+06
Will train until validation_0-rmse hasn't improved in 5 rounds.
[1]	validation_0-rmse:3.73682e+06
[2]	validation_0-rmse:3.52656e+06
[3]	validation_0-rmse:3.31749e+06
[4]	validation_0-rmse:3.13148e+06
[5]	validation_0-rmse:2.94311e+06
[6]	validation_0-rmse:2.78181e+06
[7]	validation_0-rmse:2.63442e+06
[8]	validation_0-rmse:2.50038e+06
[9]	validation_0-rmse:2.37643e+06
[10]	validation_0-rmse:2.25593e+06
[11]	validation_0-rmse:2.13376e+06
[12]	validation_0-rmse:2.02299e+06
[13]	validation_0-rmse:1.92905e+06
[14]	validation_0-rmse:1.83571e+06
[15]	validation_0-rmse:1.75626e+06
[16]	validation_0-rmse:1.66726e+06
[17]	validation_0-rmse:1.59421e+06
[18]	validation_0-rmse:1.52119e+06
[19]	validation_0-rmse:1.45806e+06
[20]	validation_0-rmse:1.39819e+06
[21]	validation_0-rmse:1.33539e+06
[22]	validation_0-rmse:1.28487e+06
[23]	validation_0-rmse:1.23651e+06
[24]	validation_0-rmse:1.19026e+06
[25]	validation_0-rmse:1.14808e+06
[26]	validation_0-rmse:1.1218e+06
[2

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0,
             importance_type='gain', learning_rate=0.07, max_delta_step=0,
             max_depth=5, min_child_weight=4, missing=None, n_estimators=500,
             n_jobs=1, nthread=2, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
             subsample=0.7, verbosity=1)

In [360]:
testY=xgbr.predict(testX)

In [398]:
xgbr = XGBRegressor(n_estimators=121,colsample_bytree= 0.7, learning_rate= 0.07, max_depth=5, min_child_weight= 4, nthread= 2, objective= 'reg:linear', silent= 1, subsample= 0.7)
testy = model_predict(xgbr,trainX,trainY,testX)
# make predictions for test data


C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\hp\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Train RMSE: 333567.0279878303


In [403]:
predictions = [round(value) if value>0 else 0 for value in testy ]
write_results(predictions,"XGBRGridCV1.xlsx")

In [364]:
result_df=pd.DataFrame(predictions)
result_df.columns=['Views']
result_df.index = testX.index
writer = ExcelWriter("XGBRGridCV.xlsx")
result_df.to_excel(writer)
writer.save()

In [365]:
!pip install lightgbm

In [392]:
import lightgbm as lgb
lgb_fit_params={"early_stopping_rounds":50, 
            "eval_metric" : 'rmse', 
            "eval_set" : [(test_x,test_y)],
            'eval_names': ['valid'],
            'verbose':100
           }

lgb_params = {'boosting_type': 'gbdt',
 'metric': 'rmse',
 'verbose': 0,
 'bagging_fraction': 0.8,
 'bagging_freq': 1,
 'feature_fraction': 0.7,
 'lambda_l1': 0.01,
 'lambda_l2': 0.01,
 'learning_rate': 0.1,
 'max_bin': 255,
 'max_depth': -1,
 'min_data_in_bin': 1,
 'min_data_in_leaf': 1,
 'num_leaves': 31}
lgb_params

clf_lgb = lgb.LGBMRegressor(n_estimators=1000, **lgb_params, random_state=5, n_jobs=2)
clf_lgb.fit(train_x, train_y, **lgb_fit_params)
clf_lgb.best_iteration_


Training until validation scores don't improve for 50 rounds
[100]	valid's rmse: 745171
[200]	valid's rmse: 735396
[300]	valid's rmse: 733203
[400]	valid's rmse: 731457
Early stopping, best iteration is:
[424]	valid's rmse: 731284


424

In [393]:
clf_lgb_full = lgb.LGBMRegressor(n_estimators=int(clf_lgb.best_iteration_*1.2), **lgb_params, random_state=5, n_jobs=2)
clf_lgb_full.fit(trainX, trainY)

LGBMRegressor(bagging_fraction=0.8, bagging_freq=1, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.7,
              importance_type='split', lambda_l1=0.01, lambda_l2=0.01,
              learning_rate=0.1, max_bin=255, max_depth=-1, metric='rmse',
              min_child_samples=20, min_child_weight=0.001, min_data_in_bin=1,
              min_data_in_leaf=1, min_split_gain=0.0, n_estimators=508,
              n_jobs=2, num_leaves=31, objective=None, random_state=5,
              reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, verbose=0)

In [395]:
ytest = model_predict(clf_lgb_full,trainX[imp_cols],trainY,testX[imp_cols])
write_results(ytest,"LightGBMR")

Train RMSE: 72964.70008252536


In [409]:
data.loc[1120309]

Name                            Queen
Genre                            rock
Timestamp                        4140
Views                             NaN
Comments                            0
Likes                               3
Popularity                          0
Followers                       70299
Followers_Per_Likes             23433
Followers_Per_Comments              0
Likes_Per_Comments                  0
Songs_Cnt                         689
Zero_Likes                          0
Zero_Comments                       1
Zero_Popularity                     1
Likes_Per_Timestamp       0.000724638
LikesnPopularity                    0
Likes_Per_Popularity                0
Mean_Popularity               1.51234
Avg_Likes                     230.435
Name: 1120309, dtype: object

In [410]:
data.loc[data.Name=='Queen']

,Name,Genre,Timestamp,Views,Comments,Likes,Popularity,Followers,Followers_Per_Likes,Followers_Per_Comments,Likes_Per_Comments,Songs_Cnt,Zero_Likes,Zero_Comments,Zero_Popularity,Likes_Per_Timestamp,LikesnPopularity,Likes_Per_Popularity,Mean_Popularity,Avg_Likes
Unique_ID,,,,,,,,,,,,,,,,,,,,
1120183,Queen,rock,2977.0,2701.0,0,8,0,70299,8787.375000,0.0,0.0,689,0,1,1,0.002687,0,0.000000,1.512337,230.435414
1120262,Queen,rock,2977.0,720.0,0,3,0,70299,23433.000000,0.0,0.0,689,0,1,1,0.001008,0,0.000000,1.512337,230.435414
1120467,Queen,rock,10282.0,63.0,0,1,0,70299,70299.000000,0.0,0.0,689,0,1,1,0.000097,0,0.000000,1.512337,230.435414
1120113,Queen,rock,2977.0,1053.0,0,11,0,70299,6390.818182,0.0,0.0,689,0,1,1,0.003695,0,0.000000,1.512337,230.435414
1119950,Queen,rock,1516.0,5023.0,0,10,2,70299,7029.900000,0.0,0.0,689,0,1,0,0.006596,20,5.000000,1.512337,230.435414
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119916,Queen,rock,129.0,NaN,0,255,13,70299,275.682353,0.0,0.0,689,0,1,0,1.976744,3315,19.615385,1.512337,230.435414
1119931,Queen,rock,857.0,NaN,0,23,5,70299,3056.478261,0.0,0.0,689,0,1,0,0.026838,115,4.600000,1.512337,230.435414
1120618,Queen,rock,10282.0,NaN,0,7,0,70299,10042.714286,0.0,0.0,689,0,1,1,0.000681,0,0.000000,1.512337,230.435414


In [411]:
import pandas_profiling

In [ ]:
pandas_profiling.ProfileReport(data)